<a href="https://colab.research.google.com/github/avrland/dataScienceFromScratch/blob/main/_Niusbot_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  #NiusBot 2021

Na podstawie:
https://minimaxir.com/2018/05/text-neural-networks/


In [1]:
%tensorflow_version 1.x
!pip install -q textgenrnn
from google.colab import files
from textgenrnn import textgenrnn
from datetime import datetime
import os

TensorFlow 1.x selected.


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


# Pobieranie bazy tytułów źródłowych
Pobieramy bazę i odczytujemy kilka pierwszych linijek, aby sprawdzić czy wszystko zostało pobrane poprawnie.

In [2]:
!pip install wget
import wget
url = 'https://raw.githubusercontent.com/avrland/dataScienceFromScratch/main/titlesNoDuplicate.txt'
wget.download(url, '/content/titles.txt')
filename = '/content/titles.txt'  
with open(filename) as fn:  
  ln = fn.readline()
  lncnt = 0
  while lncnt < 5:
       print("Line {}: {}".format(lncnt, ln.strip()))
       ln = fn.readline()
       lncnt += 1

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=52f7d865b3fce618ca258f32fd1d96c9c856000093cc60a89fbe0b515c084b73
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Line 0: 
Line 1: Polka straciła 36 tys. zł: napastnik wykiwał zarówno ją, jak i bank
Line 2: Chrome 86 na Androida pozwoli zaplanować pobieranie. Można już testować
Line 3: Poczta Polska i cyfrowa rewolucja. Identyfikacja RFID przyspieszy wysyłki
Line 4: GOG GALAXY 2.0 łączy siły z Epic Games Store. Jest wreszcie oficjalna integracja


# Parametry modelu i trenowania
Set the textgenrnn model configuration here: the default parameters here give good results for most workflows. (see the [demo notebook](https://github.com/minimaxir/textgenrnn/blob/master/docs/textgenrnn-demo.ipynb) for more information about these parameters)

If you are using an input file where documents are line-delimited, make sure to set `line_delimited` to `True`.

In [18]:
model_cfg = {
    'word_level': True,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 512,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 6,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': True,   # consider text both forwards and backward, can give a training boost
    'max_length': 10,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': True,   # set to True if each text has its own line in the source file
    'num_epochs': 40,   # set higher to train the model for longer
    'gen_epochs': 2,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.5,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [25]:
textgen.generate(10, temperature=1)

staranował " trybie solid idą stambulską . to policjant

marvel ' s avengers – premiera na mecz , gameplay , hawkeye

trzeci rekord temperatury tego odwołane

nękał można i w marvel ’ s avengers

koronawirus . 65 we francji

sony szykuje się będzie warszawą " zostanie . . . . . trwa . raport sobie takiego rzekomy "

co najmniej 63 tys . ludzi na wiecu

atmosfera wiedźmina 2 w wiedźmin 3 z the czasie rat " . zamieszanie były zabójstwa usa ?

" pettson z dubaju " : kolejny "

popis trenerskich umiejętności artety . " uczeń przechytrzył z " ? pod "



The next cell will start the actual training. And thanks to the power of Keras's CuDNN layers, training is super-fast when compared to CPU training on a local machine!

Ideally, you want a training loss less than `1.0` in order for the model to create sensible text consistently.

In [19]:
file_name = '/content/titles.txt'

#Pobierz dzisiejszą datę i godzinę
from datetime import datetime
timeNow = str(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

model_name = 'titles_20k_' + timeNow   # change to set file name of resulting trained models/texts

textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

#shutil.copy('{}_weights.hdf5'.format(model_name), '/content/drive/My Drive/model_tytuły/')
#shutil.copy('{}_vocab.json'.format(model_name), '/content/drive/My Drive/model_tytuły/')
#shutil.copy('{}_config.json'.format(model_name), '/content/drive/My Drive/model_tytuły/')

6,151 texts collected.
Training new model w/ 6-layer, 512-cell Bidirectional LSTMs
Training on 60,633 word sequences.
Epoch 1/40
59/59 [==============================] - 53s 904ms/step - loss: 80.6871
Epoch 2/40
59/59 [==============================] - 51s 869ms/step - loss: 7.9194
####################
Temperature: 0.2
####################
. . . . . . .

.

. . . . . .

####################
Temperature: 0.5
####################
duda ?



"

####################
Temperature: 1.0
####################
liczby " trzecia :

studio będą - na

liban syn najmłodszą " jerzy w i zapisy o gdańsku do kilka

Epoch 3/40
59/59 [==============================] - 52s 877ms/step - loss: 7.0039
Epoch 4/40
59/59 [==============================] - 52s 875ms/step - loss: 6.7893
####################
Temperature: 0.2
####################
" . .

" . "

" . . . "

####################
Temperature: 0.5
####################
w "

. ? , , . nie na w

.

####################
Temperature: 1.0
####################
inte

You can download a large amount of generated text from your model with the cell below! Rerun the cell as many times as you want for even more text!

In [ ]:
# this temperature schedule cycles between 1 very unexpected token, 1 unexpected token, 2 expected tokens, repeat.
# changing the temperature schedule can result in wildly different output!
temperature = [1.0, 0.5, 0.2, 0.2]   
prefix = None   # if you want each generated text to start with a given seed text

if train_cfg['line_delimited']:
  n = 1000
  max_gen_length = 60 if model_cfg['word_level'] else 300
else:
  n = 1
  max_gen_length = 2000 if model_cfg['word_level'] else 10000
  
timestring = datetime.now().strftime('%Y%m%d_%H%M%S')
gen_file = '{}_gentext_{}.txt'.format(model_name, timestring)

textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix=prefix,
                         n=n,
                         max_gen_length=max_gen_length)
files.download(gen_file)
shutil.copy('{}_weights.hdf5'.format(model_name), '/content/drive/My Drive/model/')
shutil.copy('{}_vocab.json'.format(model_name), '/content/drive/My Drive/model/')
shutil.copy('{}_config.json'.format(model_name), '/content/drive/My Drive/model/')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

You can download the weights and configuration files in the cell below, allowing you recreate the model on your own computer!

In [ ]:
files.download('{}_weights.hdf5'.format(model_name))
files.download('{}_vocab.json'.format(model_name))
files.download('{}_config.json'.format(model_name))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

To recreate the model on your own computer, after installing textgenrnn and TensorFlow, you can create a Python script with:

```
from textgenrnn import textgenrnn
textgen = textgenrnn(weights_path='newsbot1337_weights.hdf5',
                       vocab_path='newsbot1337_vocab.json',
                       config_path='newsbot1337_config.json')
                       
textgen.generate_samples(max_gen_length=100)
textgen.generate_to_file('textgenrnn_texts.txt', max_gen_length=100)
```

Have fun with your new model! :)

In [ ]:
from textgenrnn import textgenrnn
textgen = textgenrnn(weights_path='tadeusz_weights.hdf5',
                       vocab_path='tadeusz_vocab.json',
                       config_path='tadeusz_config.json')
                       
textgen.generate_samples(max_gen_length=100)
textgen.generate_to_file('textgenrnn_texts.txt', max_gen_length=100)

####################
Temperature: 0.2
####################
w

i



####################
Temperature: 0.5
####################
sędzia

sędzia

w :

####################
Temperature: 1.0
####################
w , z stawiła , są pan tylko , zląkł sam

ciotka wdzięki ogon młody na takim list domu nad paznokcie . naprzód

jak więc wszystkich dziecinna ,



# Etcetera

If the model fails to load on a local machine due to a model-size-not-matching bug (common in >30MB weights), this is due to a file export bug from Colaboratory. To work around this issue, save the weights to Google Drive with the two cells below and download from there.

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
uploaded = drive.CreateFile({'title': '{}_weights.hdf5'.format(model_name)})
uploaded.SetContentFile('{}_weights.hdf5'.format(model_name))
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1b6T6M32YnXs-c0NB-PEi6MhAdCuG7RHy


If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1